In [1]:
%cd ..

a:\Phong\Crawl_buffett


In [2]:
import pandas as pd
import os
import numpy as np
df_1 = pd.read_csv('docs/May_anhNA.csv')
df_2 = pd.read_csv('docs/may_cty.csv')

In [ ]:
df = pd.read_csv('docs/List_company_23052023 - Listing.csv')
for id in df.index:
    symbol = df['Symbol'][id]
    if df['check'][id] == 'Done':
        if os.path.exists(fr'Data\{symbol}\docs\volume.csv'):
            df['volume'][id] = 'Done'
        if os.path.exists(fr'Data\{symbol}\docs\dividend.csv'):
            df['dividend'][id] = 'Done'
        if os.path.exists(fr'Data\{symbol}\docs\checklist_get_table.csv'):
            df['table'][id] = 'Done'
df.to_csv('docs/List_company_23052023 - Listing.csv', index = False)

In [3]:
df = pd.read_csv('docs\may_anhNA.csv')
# for id in df.index:
#     symbol = df['Symbol'][id]
#     if os.path.exists(fr'Data\{symbol}\docs\check.csv'):
#         if os.path.exists(fr'Data\{symbol}\docs\volume.csv'):
#             df['volume'][id] = 'Done'
#         if os.path.exists(fr'Data\{symbol}\docs\dividend.csv'):
#             df['dividend'][id] = 'Done'
#         if os.path.exists(fr'Data\{symbol}\docs\checklist_get_table.csv'):
#             df['table'][id] = 'Done'
# df.to_csv('docs\may_anhNA.csv', index = False)
df = pd.read_csv('docs\May_anhNA.csv')
df[['volume']] = np.nan
df.to_csv('docs\May_anhNA.csv', index = False)

df = pd.read_csv('docs\may_cty.csv')
df[['volume']] = np.nan
df.to_csv('docs\may_cty.csv', index = False)

In [ ]:
df

In [ ]:
df_1[df_1['check'] == 'Done']

In [ ]:
lst_vol = list(df_1[df_1['volume'] == 'Done']['Symbol']) + list(df_2[df_2['volume'] == 'Done']['Symbol'])
len(set(lst_vol))

In [ ]:
lst_table = list(df_1[df_1['table'] == 'Done']['Symbol']) + list(df_2[df_2['table'] == 'Done']['Symbol'])
len(set(lst_table))

In [ ]:
lst_table = list(df_1[df_1['dividend'] == 'Done']['Symbol']) + list(df_2[df_2['dividend'] == 'Done']['Symbol'])
len(set(lst_table))

In [ ]:
from src.ocrPdf import ocrPDF
ocrPDF(r'tests\Data\1332\PDF\2016_Q2_決算短信(2016_11_4).pdf', 'tests/a.pdf')
from src.volume import GetVolume

gv = GetVolume()
gv.findData(path = 'tests/a.pdf')

# move file

In [1]:
%cd ..
import pandas as pd
import numpy as np
import os

a:\Phong\Crawl_buffett


In [ ]:
import shutil
PATH_DRIVER = "G:/My Drive/6_2023/Financial_1/"
PATH_IS = PATH_DRIVER + 'IncomeStatement/'
PATH_BS = PATH_DRIVER + 'BalanceSheet/'
os.makedirs(PATH_IS, exist_ok=True)
os.makedirs(PATH_BS, exist_ok=True)

symbol = 1301
def moveFinan(symbol):
    path_bs = rf'Data\{symbol}\table_bs'
    path_pl = rf'Data\{symbol}\table_pl'
    for file in os.listdir(path_bs):
        if file.endswith('Q4.csv'):
            os.makedirs(PATH_BS + '/' + str(symbol), exist_ok=True)
            shutil.copy(path_bs + '/' + file, PATH_BS + '/' + str(symbol) + '/' + file)
    for file in os.listdir(path_pl):
        if file.endswith('Q4.csv'):
            os.makedirs(PATH_IS + str(symbol), exist_ok=True)
            shutil.copy(path_pl + '/' + file, PATH_IS + '/' + str(symbol) + '/' + file)

df = pd.read_csv('docs\List_company_23052023 - Listing.csv')
for id in df.index:
    symbol = df['Symbol'][id]
    if df['check'][id] == 'Done':
        try:
            moveFinan(symbol)
        except Exception as e:
            print(symbol, e)

In [ ]:
# PATH_DRIVER = "H:/My Drive/6_2023/Done_Japan1"
PATH_DRIVER = "G:/My Drive/6_2023/DoneJapan1"
os.makedirs(rf'{PATH_DRIVER}/volume', exist_ok=True)
os.makedirs(rf'{PATH_DRIVER}/dividend', exist_ok=True)

def sortDate(df, type_ = 'volume'):
    if type_ == 'volume':
        df['Time'] = pd.to_datetime(df['Time'], format='%Y_%m_%d', errors='coerce')
    else:
        df['Time'] = pd.to_datetime(df['Time'], format='%d_%m_%Y', errors='coerce')
    df.sort_values(by=['Time'], inplace=True)
    df['Time'] = df['Time'].dt.strftime('%d/%m/%Y')
    return df

def getVolume(symbol, PATH = rf'{PATH_DRIVER}/volume/'):
    df = pd.read_csv(fr'Data\{symbol}\docs\volume.csv')
    df['volume'] = df['vol1'] - df['vol2']
    df = df[['date', 'volume']]
    df.dropna(inplace=True)
    df.rename(columns={'date': 'Time', 'volume': 'Volume'}, inplace=True)
    df = sortDate(df, type_ = 'volume')
    df.to_csv(fr'{PATH}{symbol}.csv', index = False)
    return df

def getDividend(symbol, PATH = rf'{PATH_DRIVER}/dividend/'):
    df = pd.read_csv(fr'Data\{symbol}\docs\dividend.csv')
    df_done = pd.DataFrame(columns = ['Time', 'Stock', 'Money', 'Time2'])
    df_done['Time'] = df[['time_split_Q1', 'time_split_Q2', 'time_split_Q3', 'time_split_Q4']].stack(dropna=False).reset_index(drop=True)
    df_done['Time'] = df_done['Time'].str.replace("['", '').str.replace("']", '')
    df_done['Money'] = df[['Q1', 'Q2', 'Q3', 'Q4']].stack(dropna=False).reset_index(drop=True)
    for q in  ['Q1', 'Q2', 'Q3', 'Q4']:
        df[q] = df['Year'].astype(str) + '_' + q
    df_done['Time2'] = df[['Q1', 'Q2', 'Q3', 'Q4']].stack(dropna=False).reset_index(drop=True)
    df_done.replace('－', np.nan, inplace=True)
    df_done.replace('―', np.nan, inplace=True)
    df_done.replace('-', np.nan, inplace=True)
    df_done.replace('—', np.nan, inplace=True)
    df_done.replace('一', np.nan, inplace=True)
    df_done.replace('=', np.nan, inplace=True)
    df_done.dropna(inplace=True, thresh=2)
    df_done = sortDate(df_done, type_ = 'dividend')
    df_done.to_csv(fr'{PATH}{symbol}.csv', index = False)
    return df_done

symbol = 1301
getVolume(symbol)
getDividend(symbol)

In [3]:
df = pd.read_csv('docs\List_company_23052023 - Listing.csv')
for id in df.index:
    symbol = df['Symbol'][id]
    if df['check'][id] == 'Done':
        if os.path.exists(fr'Data\{symbol}\docs\volume.csv'):
            try:
                getVolume(symbol)
            except Exception as e:
                print(symbol, e)
                break

        # if os.path.exists(fr'Data\{symbol}\docs\dividend.csv'):
        #     try:
        #         getDividend(symbol)
        #     except Exception as e:
        #         print(symbol, e)
        #         break


# volume

In [1]:
%cd ..
import pandas as pd
import numpy as np
import os
from src.volume import GetVolume

a:\Phong\Crawl_buffett


In [2]:
vol = GetVolume(path_save='')
# vol.getDataFromPdf(id_company=1333, year = 2015, quy = 'Q1')
vol.getVolume(1379)

期末発行済株式数（自己株式を含む）33,359,040株2022年３月期33,359,040株Ｑ2023年３月期１②期末自己株式数1,760,058株2022年３月期1,782,109株Ｑ2023年３月期１2022年３月期１③期中平均株式数（四半期累計）31,587,881株31,481,661株ＱＱ※四半期決算短信は公認会計士又は監査法人の四半期レビュ123,456,789,999の対象外です※業績予想の適切な利用に関する説明、その他特記事項（将来に関する記述等についてのご注意）本資料に記載されている業績見通し等の将来に関する記述は、当社が現在入手している情報及び合理的であると判断する一定の前提に基づいており、その達成を当社として約束する趣旨のものではありません。また、実際の業績等は様々な要因により大きく異なる可能性があります。業績予想の前提となる条件及び業績予想のご利用に当たっての注意事項等については、添付資料３ペ123,456,789,999ジ「１．当四半期決算に関する定性的情報（３）連結業績予想などの将来予測情報に関する説明」をご覧ください。
2--- ['202', '202']
2022_Q1: [202, 202]
期末発行済株式数（自己株式を含む）33,359,040株2021年３月期33,359,040株Ｑ2022年３月期１②期末自己株式数1,865,093株2021年３月期1,888,679株Ｑ2022年３月期１2021年３月期１③期中平均株式数（四半期累計）31,481,661株31,272,090株ＱＱ※四半期決算短信は公認会計士又は監査法人の四半期レビュ123,456,789,999の対象外です※業績予想の適切な利用に関する説明、その他特記事項（将来に関する記述等についてのご注意）本資料に記載されている業績見通し等の将来に関する記述は、当社が現在入手している情報及び合理的であると判断する一定の前提に基づいており、その達成を当社として約束する趣旨のものではありません。また、実際の業績等は様々な要因により大きく異なる可能性があります。業績予想の前提となる条件及び業績予想のご利用に当たっての注意事項等については、添付資料３ペ123,456,789,999ジ「１．当四半期決算に関する定性的情報（３）連結業績予想などの将来予測情報に関する説明」をご覧